# Segmenting and Clustering Neighborhoods in Toronto and New York

## Introduction/Business Problem

The aim of this project is to compare the neighbourhoods of New York and Toronto to pick out similariities and differences between these two cities.
In addtion, recommendations would be made on how to which city is best to live in in terms of the neighbourhood they contain.
Also, investors/business owners who want to make investments in these cities would know which business to do for each neighborhood.

In this project, I have scrapped Wikipedia to get data in Toronto and . Also, I used the Foursquare API to explore neighborhoods in Toronto. I used the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. I used the *k*-means clustering algorithm to complete this task. Finally, I used the Folium library to visualize the neighborhoods in Toronto and their emerging clusters.



In [1]:
# Importing libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

C:\Users\hp 2018\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


Libraries imported.


## Table of Contents
This capstone would be carried out in the following order

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in each City</a>

3. <a href="#item3">Analyze Each Neighborhood for each city</a>

4. <a href="#item4">Cluster Neighborhoods of each city</a>

5. <a href="#item5">Examine Clusters of each city</a>   

6. <a href="#item6">Conclusions and Recommendations</a> 
</font>
</div>

# Data Section

## 1. Download and Explore Dataset

New york has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Luckily, this dataset exists for free on the web, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

Toronto has a total of 6 boroughs and 288 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 6 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Luckily, this dataset exists for free on the web.  here is the link to the dataset: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

To get the latitude and longitudes, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

**Note:** I would have used the Geocoder package, but this package can be very unreliable.

### Loading the dataset for each city

#### For New York

In [2]:
with open('nyu_2451_34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)

In [3]:
# Taking a qucik look at the dataset
newyork_data

{'bbox': [-74.2492599487305,
  40.5033187866211,
  -73.7061614990234,
  40.9105606079102],
 'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [-73.84720052054902,
     40.89470517661],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.1',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661],
    'borough': 'Bronx',
    'name': 'Wakefield',
    'stacked': 1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.82993910812398, 40.87429419303012],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.2',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.874294193

Notice how all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [4]:
neighborhoods_data = newyork_data['features']

#### Tranform the data into a *pandas* dataframe

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
new_york = pd.DataFrame(columns=column_names)

Then let's loop through the data and fill the dataframe one row at a time.

In [6]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    new_york = new_york.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [7]:
# Examining the first 5 rows of the data for the neighborhood of New york
new_york.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


The above shows the data for neighborhoods of New York city with their latitudes and longitudes that would later be clustered.

The above shows the data for neighborhoods of New York city with their latitudes and longitudes that would later be clustered.

## Getting the data for Toronto neighborhoods

## Scraping the table from Wikipedia

In [8]:
# Importing requests
import requests

In [9]:
# Scraping the table from Wikipedia
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [10]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xc7FDApAADoAACwNUTAAAABC","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":926306543,"wgRevisionId":926306543,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communi

In [11]:
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

### Creating the dataframe from the scraped data

In [12]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

In [13]:
# Viewwing the first 5 rows of the data frame
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287 entries, 1 to 287
Data columns (total 3 columns):
PostalCode      287 non-null object
Borough         287 non-null object
Neighborhood    287 non-null object
dtypes: object(3)
memory usage: 5.6+ KB


### Observations

Clearly, there are some Borough that are not assigned. I decided to remove such borough

In [15]:
df[df['Borough'] == 'Not assigned'].describe()

,PostalCode,Borough,Neighborhood
count,77,77,77
unique,77,1,1
top,M6T,Not assigned,Not assigned
freq,1,77,77


### Observations 

From the information about the data, there are 288 rows. However, I saw that 77 observations had Boroughs as not assigned.

Therefore, I dropped those rows and we have 211 rows left

In [16]:
# delete all rows with column 'Borough' has value Not assigned
names = df[ df['Borough'] == 'Not assigned'].index
names
df.drop(names , inplace=True)

In [17]:
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor


## Joining Neighborhoods with same Postal Code Area

For example, in the table on the Wikipedia page, one can see that **M5A** is listed twice and has two neighborhoods: **Harbourfront** and **Regent Park.** These two rows will be combined into one row with the neighborhoods separated with a comma.

In [18]:
#Combining two rows into one row with the neighborhoods separated with a comma
d = {'Borough': 'first','Neighborhood': ','.join}
df_new = df.groupby('PostalCode', as_index=False).aggregate(d).reindex(columns=df.columns)
print(df_new)

    PostalCode           Borough  \
0          M1B       Scarborough   
1          M1C       Scarborough   
2          M1E       Scarborough   
3          M1G       Scarborough   
4          M1H       Scarborough   
5          M1J       Scarborough   
6          M1K       Scarborough   
7          M1L       Scarborough   
8          M1M       Scarborough   
9          M1N       Scarborough   
10         M1P       Scarborough   
11         M1R       Scarborough   
12         M1S       Scarborough   
13         M1T       Scarborough   
14         M1V       Scarborough   
15         M1W       Scarborough   
16         M1X       Scarborough   
17         M2H        North York   
18         M2J        North York   
19         M2K        North York   
20         M2L        North York   
21         M2M        North York   
22         M2N        North York   
23         M2P        North York   
24         M2R        North York   
25         M3A        North York   
26         M3B        North 

In [19]:
# Putting in a dataframe
df_new = pd.DataFrame(df_new)

In [20]:
df_new

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [21]:
# Checking the shape of the data
df_new.shape

(103, 3)

## Merging the Latitudes and Longitudes to the dataframe above

Since my scrapped data is properly cleaned. I can add the csv file containing the latitudes and longitudes.

In [22]:
# Loading the CSV file
geo_data = pd.read_csv('Geospatial_Coordinates.csv')

In [23]:
# Dropping the postal code column in the CSV file
geo_data.drop('Postal Code', axis=1, inplace = True)

In [24]:
# This shows that the CSV file was properly loaded
geo_data.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


In [25]:
# I checked the shape to make sure it has the same number of rows (103) with the data I scrapped and cleaned.
# Since they have the same shape. I can merge them

geo_data.shape

(103, 2)

In [26]:
# Merging the cleaned scrapped data and the cleaned Geo Spatial Data
geo_data_new = pd.concat([df_new, geo_data], axis =1)

In [27]:
# Viewing the first five rows of the data
geo_data_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### Create a map of Toronto with neighborhoods superimposed on top.

In [28]:
# Toronto Latitude and Longitude
latitude = 43.651070
longitude = -79.347015

In [29]:
# create map of Toronto using latitude and longitude values
map_toron = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(geo_data_new['Latitude'], geo_data_new['Longitude'], geo_data_new['Borough'], 
                                           geo_data_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toron)  
    
map_toron
#map_toron.save('toronto_neigh_map.html') # Saving the map for viewing later 

**Folium** is a great visualization library. Clicking on each circle mark reveals the name of the neighborhood and its respective borough.

However, for illustration purposes, I simplify the above map and segment and cluster only the neighborhoods in Scarborough. So I sliced the original dataframe and create a new dataframe of the Scarborough data.

In [30]:
scar_data = geo_data_new[geo_data_new['Borough'] == 'Scarborough'].reset_index(drop=True)
scar_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


As I did with all of Toronto, I visualize Scarborough and the neighborhoods in it.

In [31]:
# Scarborough Latitude and Longitude
latitude = 43.777702
longitude = -79.233238

In [32]:
# create map of Scarborough using latitude and longitude values
map_scar = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(geo_data_new['Latitude'], geo_data_new['Longitude'], geo_data_new['Borough'], 
                                           geo_data_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scar)  
    
map_scar
#map_scar.save('Scarborough_neigh_map.html') # Saving the map of Scarborough for viewing later.

## 2. Employing Foursquare API

Next, I start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define my Foursquare Credentials and Version

In [33]:
CLIENT_ID = 'KS0GCILNWWLOOOOTCLGKSVSEWDHCMNLSDDNS1TX1MEITC1C3' # my Foursquare ID
CLIENT_SECRET = 'ACHJPQK2UUDDBV4S1R3JBCIOLDSMFEZN3TI3CFV33ZXLGU3F' # my Foursquare Secret
VERSION = '20180604'
LIMIT = 30
radius = 500

#### Exploring the first neighborhood in our dataframe.

In [34]:
# Get the Neighbourhood's name
scar_data.loc[0, 'Neighborhood']

'Rouge,Malvern'

In [35]:
# Get the neighbourhood's lotitude and longitude values
neighborhood_latitude = scar_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scar_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scar_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


#### Now, I got the top 100 venues that are in Rouge,Malvern within a radius of 5000 meters.

In [36]:
# Getting the URL
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=KS0GCILNWWLOOOOTCLGKSVSEWDHCMNLSDDNS1TX1MEITC1C3&client_secret=ACHJPQK2UUDDBV4S1R3JBCIOLDSMFEZN3TI3CFV33ZXLGU3F&v=20180604&ll=43.806686299999996,-79.19435340000001&radius=5000&limit=100'

In [37]:
#Send the GET requests and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ddad931c546f3001b44a235'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-542858a0498e22b7cfa91070-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/sports_outdoors_',
          'suffix': '.png'},
         'id': '4f4528bc4b90abdf24c9de85',
         'name': 'Athletics & Sports',
         'pluralName': 'Athletics & Sports',
         'primary': True,
         'shortName': 'Athletics & Sports'}],
       'id': '542858a0498e22b7cfa91070',
       'location': {'address': '875 Morningside Ave',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 1788,
        'formattedAddress': ['875 Morningside Ave',
         'Toronto ON M1C 0C7',
         'Canada'],
        'labeledLatLngs': [{'label': 'disp

In [38]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Cleaning the json and structure it into a *pandas* dataframe.

In [39]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Toronto Pan Am Sports Centre,Athletics & Sports,43.790623,-79.193869
1,African Rainforest Pavilion,Zoo Exhibit,43.817725,-79.183433
2,Toronto Zoo,Zoo,43.820582,-79.181551
3,Polar Bear Exhibit,Zoo,43.823372,-79.185145
4,Canadiana exhibit,Zoo Exhibit,43.817962,-79.193374


### And how many venues were returned by Foursquare?

In [40]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## Explore Neighborhoods in Scarborough

#### Creating a function to repeat the same process to all the neighborhoods in Scarborough

In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Creating a function on each neighborhood and create a new dataframe called *Scarborough_venues*.

In [42]:

scar_venues = getNearbyVenues(names=scar_data['Neighborhood'],
                                   latitudes=scar_data['Latitude'],
                                   longitudes=scar_data['Longitude']
                                  )


Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge


In [43]:
# Checking how many venues were returned for each neighborhood
scar_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,100,100,100,100,100,100
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",100,100,100,100,100,100
"Birch Cliff,Cliffside West",100,100,100,100,100,100
Cedarbrae,100,100,100,100,100,100
"Clairlea,Golden Mile,Oakridge",100,100,100,100,100,100
"Clarks Corners,Sullivan,Tam O'Shanter",100,100,100,100,100,100
"Cliffcrest,Cliffside,Scarborough Village West",100,100,100,100,100,100
"Dorset Park,Scarborough Town Centre,Wexford Heights",100,100,100,100,100,100
"East Birchmount Park,Ionview,Kennedy Park",100,100,100,100,100,100


#### How many unique categories can be curated from all the returned venues?

In [44]:
print('There are {} uniques categories.'.format(len(scar_venues['Venue Category'].unique())))

There are 147 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [45]:
# one hot encoding
scar_onehot = pd.get_dummies(scar_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scar_onehot['Neighborhood'] = scar_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scar_onehot.columns[-1]] + list(scar_onehot.columns[:-1])
scar_onehot = scar_onehot[fixed_columns]

scar_onehot.head()

,Zoo Exhibit,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Beach,Beer Store,Big Box Store,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Butcher,Café,Campground,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Dry Cleaner,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Hardware Store,Health Food Store,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hotel,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Liquor Store,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Store,Nail Salon,National Park,Neighborhood,Noodle House,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Pool Hall,Portuguese Restaurant,Pub,Restaurant,Rock Climbing Spot,Sandwich Place,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Smoothie Shop,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Yoga Studio,Zoo
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
# Examining the new dataframe size.
scar_onehot.shape

(1689, 147)

#### Next, grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [47]:
scar_grouped = scar_onehot.groupby('Neighborhood').mean().reset_index()
scar_grouped

,Neighborhood,Zoo Exhibit,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Beach,Beer Store,Big Box Store,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Butcher,Café,Campground,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Design Studio,Dessert Shop,Diner,Discount Store,Dog Run,Dry Cleaner,Dumpling Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Hardware Store,Health Food Store,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hotel,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Liquor Store,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Music Store,Nail Salon,National Park,Noodle House,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Pool Hall,Portuguese Restaurant,Pub,Restaurant,Rock Climbing Spot,Sandwich Place,Science Museum,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skating Rink,Smoothie Shop,Snack Place,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Yoga Studio,Zoo
0,Agincourt,0.000000,0.00,0.01,0.000000,0.01,0.01,0.000000,0.00,0.00,0.00,0.00,0.030000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.02,0.00,0.010000,0.00,0.00,0.00,0.020000,0.01,0.00,0.000000,0.000000,0.02,0.050000,0.100000,0.00,0.01,0.00,0.050000,0.00,0.00,0.000000,0.00,0.00,0.00,0.010000,0.00,0.000000,0.00,0.000000,0.01,0.00,0.00,0.01,0.02,0.00,0.00,0.020000,0.00,0.00,0.00,0.01,0.00,0.000000,0.00,0.010000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.02,0.010000,0.02,0.01,0.02,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.000000,0.04,0.00,0.010000,0.010000,0.000000,0.01,0.010000,0.00,0.00,0.010000,0.00,0.01,0.00,0.00,0.00,0.000000,0.04,0.00,0.000000,0.00,0.030000,0.00,0.01,0.010000,0.030000,0.00,0.00,0.01,0.010000,0.010000,0.00,0.030000,0.00,0.01,0.00,0.00,0.00,0.000000,0.01,0.00,0.000000,0.010000,0.00,0.00,0.01,0.02,0.030000,0.03,0.01,0.00,0.010000,0.01,0.00,0.00,0.00,0.01,0.00,0.02,0.00,0.000000,0.00,0.000000
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.00,0.00,0.000000,0.01,0.02,0.000000,0.01,0.01,0.00,0.00,0.060000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.01,0.00,0.020000,0.00,0.05,0.00,0.010000,0.00,0.00,0.000000,0.000000,0.01,0.050000,0.120000,0.01,0.00,0.00,0.010000,0.00,0.00,0.000000,0.00,0.00,0.00,0.020000,0.00,0.000000,0.00,0.000000,0.01,0.00,0.00,0.01,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.010000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.01,0.020000,0.01,0.01,0.02,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.01,0.00,0.010000,0.04,0.00,0.010000,0.030000,0.010000,0.03,0.000000,0.01,0.00,0.010000,0.00,0.00,0.01,0.00,0.00,0.000000,0.03,0.01,0.000000,0.00,0.010000,0.00,0.00,0.020000,0.010000,0.00,0.00,0.00,0.000000,0.010000,0.00,0.010000,0.00,0.00,0.00,0.01,0.00,0.000000,0.01,0.00,0.000000,0.010000,0.00,0.01,0.01,0.01,0.040000,0.02,0.01,0.01,0.000000,0.01,0.00,0.00,0.00,0.02,0.00,0.02,0.00,0.000000,0.00,0.000000
2,"Birch 

In [48]:
# Confirming the new size
scar_grouped.shape

(17, 147)

#### Printing each neighborhood along with the top 5 most common venues

In [49]:
num_top_venues = 5

for hood in scar_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scar_grouped[scar_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.10
1           Coffee Shop  0.05
2  Caribbean Restaurant  0.05
3          Noodle House  0.04
4     Indian Restaurant  0.04


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                  venue  freq
0    Chinese Restaurant  0.12
1                Bakery  0.06
2  Caribbean Restaurant  0.05
3       Bubble Tea Shop  0.05
4     Indian Restaurant  0.04


----Birch Cliff,Cliffside West----
            venue  freq
0           Beach  0.08
1     Coffee Shop  0.06
2            Park  0.06
3  Breakfast Spot  0.04
4          Bakery  0.04


----Cedarbrae----
                  venue  freq
0           Coffee Shop  0.10
1  Fast Food Restaurant  0.09
2        Sandwich Place  0.06
3           Pizza Place  0.05
4     Indian Restaurant  0.05


----Clairlea,Golden Mile,Oakridge----
                       venue  freq
0                       Park  0.06
1  Middle Eastern Restaurant  0.05
2                Coffee Shop  0.

#### Putting that into a *pandas* dataframe
First, writing a function to sort the venues in descending order.

In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now, creating the new dataframe and display the top 10 venues for each neighborhood.

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scar_grouped['Neighborhood']

for ind in np.arange(scar_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scar_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Coffee Shop,Caribbean Restaurant,Indian Restaurant,Noodle House,Park,Sushi Restaurant,Bakery,Pizza Place,Sandwich Place
1,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Chinese Restaurant,Bakery,Bubble Tea Shop,Caribbean Restaurant,Supermarket,Indian Restaurant,Korean Restaurant,Japanese Restaurant,Noodle House,Vegetarian / Vegan Restaurant
2,"Birch Cliff,Cliffside West",Beach,Coffee Shop,Park,Breakfast Spot,Bakery,Café,Gastropub,Ice Cream Shop,Liquor Store,BBQ Joint
3,Cedarbrae,Coffee Shop,Fast Food Restaurant,Sandwich Place,Indian Restaurant,Pizza Place,Caribbean Restaurant,Park,Chinese Restaurant,Fried Chicken Joint,Steakhouse
4,"Clairlea,Golden Mile,Oakridge",Park,Middle Eastern Restaurant,Coffee Shop,Beach,Café,Breakfast Spot,Gastropub,Bakery,Supermarket,Burger Joint


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [52]:
# set number of clusters
kclusters = 5

scar_grouped_clustering = scar_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scar_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 4, 3, 4, 1, 3, 2, 2, 3])

#### Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [53]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scar_merged = scar_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scar_merged = scar_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scar_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,0,Zoo Exhibit,Coffee Shop,Pharmacy,Sandwich Place,Fast Food Restaurant,Gas Station,Pizza Place,Burger Joint,Fried Chicken Joint,Breakfast Spot
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0,Zoo Exhibit,Coffee Shop,Park,Breakfast Spot,Gas Station,Pharmacy,Pizza Place,Smoothie Shop,Beer Store,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,3,Coffee Shop,Pharmacy,Sandwich Place,Fast Food Restaurant,Beer Store,Park,Indian Restaurant,Gym,Breakfast Spot,Gas Station
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3,Coffee Shop,Fast Food Restaurant,Pizza Place,Burger Joint,Caribbean Restaurant,Pharmacy,Park,Sandwich Place,Breakfast Spot,Indian Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3,Coffee Shop,Fast Food Restaurant,Sandwich Place,Indian Restaurant,Pizza Place,Caribbean Restaurant,Park,Chinese Restaurant,Fried Chicken Joint,Steakhouse


### Finally, visualizing the resulting clusters

In [54]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scar_merged['Latitude'], scar_merged['Longitude'], scar_merged['Neighborhood'], scar_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
#map_clusters.save('Clustered_Scarborough_Neighborhood_Map.html') # Saving the map of clustered Scarborough for viewing later.

<a id='item5'></a>

## 5. Examine Clusters

Now, one can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, one can then assign a name to each cluster.

#### Cluster 1

In [55]:
scar_merged.loc[scar_merged['Cluster Labels'] == 0, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Zoo Exhibit,Coffee Shop,Pharmacy,Sandwich Place,Fast Food Restaurant,Gas Station,Pizza Place,Burger Joint,Fried Chicken Joint,Breakfast Spot
1,Scarborough,0,Zoo Exhibit,Coffee Shop,Park,Breakfast Spot,Gas Station,Pharmacy,Pizza Place,Smoothie Shop,Beer Store,Fast Food Restaurant
16,Scarborough,0,Zoo Exhibit,Coffee Shop,Sandwich Place,Fast Food Restaurant,Pharmacy,Pizza Place,Gas Station,Hakka Restaurant,Grocery Store,Burger Joint


Cluster One can be named **Chinese Restaurant** because the most common value in the first cluster is the **Chinese Restaurant**. Well, who knows, there could be more Chinese staying or living around those neighborhoods.

#### Cluster 2

In [56]:
scar_merged.loc[scar_merged['Cluster Labels'] == 1, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,1,Chinese Restaurant,Coffee Shop,Caribbean Restaurant,Indian Restaurant,Noodle House,Park,Sushi Restaurant,Bakery,Pizza Place,Sandwich Place
13,Scarborough,1,Chinese Restaurant,Caribbean Restaurant,Bakery,Supermarket,Middle Eastern Restaurant,Coffee Shop,Indian Restaurant,Noodle House,Korean Restaurant,Japanese Restaurant
14,Scarborough,1,Chinese Restaurant,Bakery,Bubble Tea Shop,Caribbean Restaurant,Supermarket,Indian Restaurant,Korean Restaurant,Japanese Restaurant,Noodle House,Vegetarian / Vegan Restaurant
15,Scarborough,1,Chinese Restaurant,Bakery,Bubble Tea Shop,Caribbean Restaurant,Japanese Restaurant,Korean Restaurant,Supermarket,Noodle House,Asian Restaurant,Vietnamese Restaurant


In cluster two as seen above, the most common venue is the **Coffee Shop**. This means there are more coffee shops, and that is why the cluster is grouped this way. It is possible that most people in this neighborhood takes coffee.

#### Cluster 3

In [57]:
scar_merged.loc[scar_merged['Cluster Labels'] == 2, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,2,Coffee Shop,Burger Joint,Park,Chinese Restaurant,Supermarket,Sandwich Place,Steakhouse,Burrito Place,Indian Restaurant,Fast Food Restaurant
6,Scarborough,2,Coffee Shop,Park,Middle Eastern Restaurant,Burger Joint,Chinese Restaurant,Gym,Indian Restaurant,Bakery,Supermarket,Burrito Place
10,Scarborough,2,Coffee Shop,Indian Restaurant,Middle Eastern Restaurant,Caribbean Restaurant,Supermarket,Pizza Place,Burger Joint,Chinese Restaurant,Pharmacy,Asian Restaurant
11,Scarborough,2,Middle Eastern Restaurant,Coffee Shop,Supermarket,Burrito Place,Mediterranean Restaurant,Burger Joint,Indian Restaurant,Caribbean Restaurant,Pizza Place,Pet Store


Cluster 3 does not clearly have a distinctive characteristics differentiating the cluster. However, I can conclude and name this cluster **Relaxation Venues** because it occurs to me that most venues here are for relaxation (looking at the parks, Beach etc)

#### Cluster 4

In [58]:
scar_merged.loc[scar_merged['Cluster Labels'] == 3, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,3,Coffee Shop,Pharmacy,Sandwich Place,Fast Food Restaurant,Beer Store,Park,Indian Restaurant,Gym,Breakfast Spot,Gas Station
3,Scarborough,3,Coffee Shop,Fast Food Restaurant,Pizza Place,Burger Joint,Caribbean Restaurant,Pharmacy,Park,Sandwich Place,Breakfast Spot,Indian Restaurant
4,Scarborough,3,Coffee Shop,Fast Food Restaurant,Sandwich Place,Indian Restaurant,Pizza Place,Caribbean Restaurant,Park,Chinese Restaurant,Fried Chicken Joint,Steakhouse
8,Scarborough,3,Coffee Shop,Pharmacy,Sandwich Place,Burger Joint,Grocery Store,Gym,Park,Fast Food Restaurant,Beer Store,Clothing Store


For cluster 4, **Zoo Exhibit** happen to be the most common venue. This tells me that people can go on excursion here and still find places to relax like the parks and coffee shops.

#### Cluster 5

In [59]:
scar_merged.loc[scar_merged['Cluster Labels'] == 4, scar_merged.columns[[1] + list(range(5, scar_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Scarborough,4,Park,Middle Eastern Restaurant,Coffee Shop,Beach,Café,Breakfast Spot,Gastropub,Bakery,Supermarket,Burger Joint
9,Scarborough,4,Beach,Coffee Shop,Park,Breakfast Spot,Bakery,Café,Gastropub,Ice Cream Shop,Liquor Store,BBQ Joint


Cluster five occurs to me to be more of restaurants or eateries just like cluster one.